In [83]:
import csv, json, operator
from collections import defaultdict, Counter
from nltk.tokenize import RegexpTokenizer
import nltk
import math

In [117]:
labels = ['All', 'Trump', 'NoTrump']
train_data = list()
train_labels = list()
data_source = {'realDonaldTrump': 12322,
             'SecretaryCarson': 1242,
             'MartinOMalley': 2211,
             'JebBush': 2757,
             'BarackObama': 2880,
             'HillaryClinton': 1774,
             'BernieSanders': 1458}
text_length = {key: defaultdict(int) for key in labels}
word_level = {key: defaultdict(int) for key in labels}
unigram_count = {key: defaultdict(int) for key in labels}
bigram_count = {key: defaultdict(int) for key in labels}
trigram_count = {key: defaultdict(int) for key in labels}
dev_data = list()
dev_labels = list()

## Collect statistics
TODO: word difficulty classification

In [118]:
all_label = 'All'
phrases = ['uni', 'bi', 'tri']
tokenizer = RegexpTokenizer(r'\w+')
Trump_vocab = {key: defaultdict(int) for key in phrases}
noTrump_vocab = {key: defaultdict(int) for key in phrases}
line_count = 0
with open('train.csv', newline='', encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')    
    for row in csv_reader:
        text = row[0]
        label = row[1]
        train_data.append(text)
        train_labels.append(label)        
        text_length[all_label][int(math.ceil(len(text) / 10.0)) * 10] += 1
        text_length[label][int(math.ceil(len(text) / 10.0)) * 10] += 1                        
        words = tokenizer.tokenize(row[0])
        for unigram in words:
            unigram_count[label][unigram] += 1
            unigram_count[all_label][unigram] += 1
        for bigram in list(nltk.bigrams(words)):
            bigram_count[label][bigram] += 1
            bigram_count[all_label][bigram] += 1
        for trigram in list(nltk.trigrams(words)):
            trigram_count[label][trigram] += 1
            trigram_count[all_label][trigram] += 1
            
with open('dev.csv', newline='', encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')    
    for row in csv_reader:                                
        dev_data.append(row[0])
        dev_labels.append(row[1])        

In [119]:
from collections import Counter
import itertools
label_counter = {'total': len(train_data), 'validation': len(dev_data)}
label_counter.update(Counter(train_labels).most_common())
for k, v in text_length.items():
    text_length[k] = {vk: vv for (vk, vv) in sorted(v.items())}
for k, v in unigram_count.items():
    unigram_count[k] = {vk: vv for (vk, vv) in sorted(v.items(), key=operator.itemgetter(1), reverse=True)}
    unigram_count[k] = dict(itertools.islice(unigram_count[k].items(), 1000))
for k, v in bigram_count.items():
    bigram_count[k] = {' '.join(vk): vv for (vk, vv) in sorted(v.items(), key=operator.itemgetter(1), reverse=True)}
    bigram_count[k] = dict(itertools.islice(bigram_count[k].items(), 1000))
for k, v in trigram_count.items():
    trigram_count[k] = {' '.join(vk): vv for (vk, vv) in sorted(v.items(), key=operator.itemgetter(1), reverse=True)}
    trigram_count[k] = dict(itertools.islice(trigram_count[k].items(), 1000))
basic_dict = {'count': label_counter, 'length': text_length, 'source': data_source,
              'unigram': unigram_count, 'bigram': bigram_count, 'trigram': trigram_count }            

1000

In [122]:
with open('dataset_statistic.json', 'w') as fp:
    json.dump(basic_dict, fp)